**Uber Data Walkthrough Project**

**1. Introduction**<br>
In this project, I aim to thoroughly analyze Uber data to gain a comprehensive understanding. This will enable us to refine the data and explore its potential applications effectively.<br>
**2. Data Collection**<br>
*Data Source:* You can get the data from <a href = "https://www.kaggle.com/datasets/zusmani/uberdrives"> here </a><br>
**3. Methodology**<br>
*Data Cleaning:* And this is the objective we will address in this notebook.

In [3]:
#Importing liberaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

Let's have a look on the data we have.

In [4]:
df = pd.read_csv('UberDataset.csv')
df

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE
0,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain
1,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,5.0,NaN
2,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies
3,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,4.7,Meeting
4,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit
...,...,...,...,...,...,...,...
1151,12/31/2016 13:24,12/31/2016 13:42,Business,Kar?chi,Unknown Location,3.9,Temporary Site
1152,12/31/2016 15:03,12/31/2016 15:38,Business,Unknown Location,Unknown Location,16.2,Meeting
1153,12/31/2016 21:32,12/31/2016 21:50,Business,Katunayake,Gampaha,6.4,Temporary Site
1154,12/31/2016 22:08,12/31/2016 23:51,Business,Gampaha,Ilukwatta,48.2,Temporary Site


In [5]:
df.describe()

,MILES
count,1156.000000
mean,21.115398
std,359.299007
min,0.500000
25%,2.900000
50%,6.000000
75%,10.400000
max,12204.700000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   START_DATE  1156 non-null   object 
 1   END_DATE    1155 non-null   object 
 2   CATEGORY    1155 non-null   object 
 3   START       1155 non-null   object 
 4   STOP        1155 non-null   object 
 5   MILES       1156 non-null   float64
 6   PURPOSE     653 non-null    object 
dtypes: float64(1), object(6)
memory usage: 63.3+ KB


One of the most important things to do is checking missing values.

In [7]:
df.isna().sum()

START_DATE      0
END_DATE        1
CATEGORY        1
START           1
STOP            1
MILES           0
PURPOSE       503
dtype: int64

***We have here some problems***
1. The date itself is not particularly significant, as Uber trips typically do not exceed one day. Therefore, the primary information we need from the date columns is the start and end times of the trips, which we will later refer to as "DURATION in minutes".

In [8]:
df['START_DATE']

0       01-01-2016 21:11
1       01-02-2016 01:25
2       01-02-2016 20:25
3       01-05-2016 17:31
4       01-06-2016 14:42
              ...       
1151    12/31/2016 13:24
1152    12/31/2016 15:03
1153    12/31/2016 21:32
1154    12/31/2016 22:08
1155              Totals
Name: START_DATE, Length: 1156, dtype: object

In [9]:
df['START_TIME'] = df['START_DATE'].str.split(' ').str[1]
df['END_TIME'] = df['END_DATE'].str.split(' ').str[1]

Here, now it looks better, and it's more clear. now we have "START_TIME" and "END_TIME".

In [10]:
df = df.drop('START_DATE', axis=1)
df = df.drop('END_DATE', axis=1)
df

,CATEGORY,START,STOP,MILES,PURPOSE,START_TIME,END_TIME
0,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain,21:11,21:17
1,Business,Fort Pierce,Fort Pierce,5.0,NaN,01:25,01:37
2,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies,20:25,20:38
3,Business,Fort Pierce,Fort Pierce,4.7,Meeting,17:31,17:45
4,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit,14:42,15:49
...,...,...,...,...,...,...,...
1151,Business,Kar?chi,Unknown Location,3.9,Temporary Site,13:24,13:42
1152,Business,Unknown Location,Unknown Location,16.2,Meeting,15:03,15:38
1153,Business,Katunayake,Gampaha,6.4,Temporary Site,21:32,21:50
1154,Business,Gampaha,Ilukwatta,48.2,Temporary Site,22:08,23:51


2. There's no "NULL" data but there's "Unknown Location" which is exactly the same thing. It's already not valid data and this will affect our data.

In [11]:
stop_unknown = (df['STOP'] == 'Unknown Location').sum()
start_unknown = (df['START'] == 'Unknown Location').sum()
print("Unknown Stop Location:", stop_unknown)
print("Unknown Start Location:", start_unknown)

Unknown Stop Location: 149
Unknown Start Location: 148


You will see the way I handeled this problem by dropping the "Unknown Location", we will get there in a moment.

We can drop the other "NULL" values in the other columns, it's not that much. You can take a look back up and you will see that the "NULL" values in these columns are so little.

In [12]:
df = df.dropna(subset=['CATEGORY', 'END_TIME', 'STOP'])
df

,CATEGORY,START,STOP,MILES,PURPOSE,START_TIME,END_TIME
0,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain,21:11,21:17
1,Business,Fort Pierce,Fort Pierce,5.0,NaN,01:25,01:37
2,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies,20:25,20:38
3,Business,Fort Pierce,Fort Pierce,4.7,Meeting,17:31,17:45
4,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit,14:42,15:49
...,...,...,...,...,...,...,...
1150,Business,Kar?chi,Kar?chi,0.7,Meeting,1:07,1:14
1151,Business,Kar?chi,Unknown Location,3.9,Temporary Site,13:24,13:42
1152,Business,Unknown Location,Unknown Location,16.2,Meeting,15:03,15:38
1153,Business,Katunayake,Gampaha,6.4,Temporary Site,21:32,21:50


3. Creating a "DURATION in minutes" column.

In [13]:
df['START_TIME'] = pd.to_datetime(df['START_TIME'], format='%H:%M')
df['END_TIME'] = pd.to_datetime(df['END_TIME'], format='%H:%M')
df['DURATION'] = df['END_TIME'] - df['START_TIME']
df['DURATION'] = df['DURATION'].apply(lambda x: x.total_seconds() / 60)
df

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13256\3741521045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['START_TIME'] = pd.to_datetime(df['START_TIME'], format='%H:%M')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13256\3741521045.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['END_TIME'] = pd.to_datetime(df['END_TIME'], format='%H:%M')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13256\3741521045.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,CATEGORY,START,STOP,MILES,PURPOSE,START_TIME,END_TIME,DURATION
0,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain,1900-01-01 21:11:00,1900-01-01 21:17:00,6.0
1,Business,Fort Pierce,Fort Pierce,5.0,NaN,1900-01-01 01:25:00,1900-01-01 01:37:00,12.0
2,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies,1900-01-01 20:25:00,1900-01-01 20:38:00,13.0
3,Business,Fort Pierce,Fort Pierce,4.7,Meeting,1900-01-01 17:31:00,1900-01-01 17:45:00,14.0
4,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit,1900-01-01 14:42:00,1900-01-01 15:49:00,67.0
...,...,...,...,...,...,...,...,...
1150,Business,Kar?chi,Kar?chi,0.7,Meeting,1900-01-01 01:07:00,1900-01-01 01:14:00,7.0
1151,Business,Kar?chi,Unknown Location,3.9,Temporary Site,1900-01-01 13:24:00,1900-01-01 13:42:00,18.0
1152,Business,Unknown Location,Unknown Location,16.2,Meeting,1900-01-01 15:03:00,1900-01-01 15:38:00,35.0
1153,Business,Katunayake,Gampaha,6.4,Temporary Site,1900-01-01 21:32:00,1900-01-01 21:50:00,18.0


So from here we can see that "PURPOSE" column is not that usefull for our data.

In [14]:
df['PURPOSE'].unique()

array(['Meal/Entertain', nan, 'Errand/Supplies', 'Meeting',
       'Customer Visit', 'Temporary Site', 'Between Offices',
       'Charity ($)', 'Commute', 'Moving', 'Airport/Travel'], dtype=object)

Now let's shape our data in the way we want it!

In [15]:
df = df.drop('START_TIME', axis=1)
df = df.drop('END_TIME', axis=1)
df = df.drop('PURPOSE', axis=1)
df

,CATEGORY,START,STOP,MILES,DURATION
0,Business,Fort Pierce,Fort Pierce,5.1,6.0
1,Business,Fort Pierce,Fort Pierce,5.0,12.0
2,Business,Fort Pierce,Fort Pierce,4.8,13.0
3,Business,Fort Pierce,Fort Pierce,4.7,14.0
4,Business,Fort Pierce,West Palm Beach,63.7,67.0
...,...,...,...,...,...
1150,Business,Kar?chi,Kar?chi,0.7,7.0
1151,Business,Kar?chi,Unknown Location,3.9,18.0
1152,Business,Unknown Location,Unknown Location,16.2,35.0
1153,Business,Katunayake,Gampaha,6.4,18.0


In [16]:
df.rename(columns={'DURATION': 'DURATION in minutes'}, inplace=True)
df

,CATEGORY,START,STOP,MILES,DURATION in minutes
0,Business,Fort Pierce,Fort Pierce,5.1,6.0
1,Business,Fort Pierce,Fort Pierce,5.0,12.0
2,Business,Fort Pierce,Fort Pierce,4.8,13.0
3,Business,Fort Pierce,Fort Pierce,4.7,14.0
4,Business,Fort Pierce,West Palm Beach,63.7,67.0
...,...,...,...,...,...
1150,Business,Kar?chi,Kar?chi,0.7,7.0
1151,Business,Kar?chi,Unknown Location,3.9,18.0
1152,Business,Unknown Location,Unknown Location,16.2,35.0
1153,Business,Katunayake,Gampaha,6.4,18.0


Here! Now we have our data in shape in the way that we needed.

Now let's fix this "Unknown Location" problem.

In [17]:
unknown_start_count = (df['START'] == 'Unknown Location').sum()
print("Number of 'Unknown Location' values in the START row:", unknown_start_count)
unknown_stop_count = (df['STOP'] == 'Unknown Location').sum()
print("Number of 'Unknown Location' values in the STOP row:", unknown_stop_count)


Number of 'Unknown Location' values in the START row: 148
Number of 'Unknown Location' values in the STOP row: 149


In [18]:
df = df[(df['START'] != 'Unknown Location') & (df['STOP'] != 'Unknown Location')]
df

,CATEGORY,START,STOP,MILES,DURATION in minutes
0,Business,Fort Pierce,Fort Pierce,5.1,6.0
1,Business,Fort Pierce,Fort Pierce,5.0,12.0
2,Business,Fort Pierce,Fort Pierce,4.8,13.0
3,Business,Fort Pierce,Fort Pierce,4.7,14.0
4,Business,Fort Pierce,West Palm Beach,63.7,67.0
...,...,...,...,...,...
1148,Business,Kar?chi,Kar?chi,4.6,23.0
1149,Business,Kar?chi,Kar?chi,0.8,4.0
1150,Business,Kar?chi,Kar?chi,0.7,7.0
1153,Business,Katunayake,Gampaha,6.4,18.0


It's clean now!!

In [19]:
unknown_start_count = (df['START'] == 'Unknown Location').sum()
print("Number of 'Unknown Location' values in the START row:", unknown_start_count)
unknown_stop_count = (df['STOP'] == 'Unknown Location').sum()
print("Number of 'Unknown Location' values in the STOP row:", unknown_stop_count)

Number of 'Unknown Location' values in the START row: 0
Number of 'Unknown Location' values in the STOP row: 0


Final look to our clean data.

In [20]:
df

,CATEGORY,START,STOP,MILES,DURATION in minutes
0,Business,Fort Pierce,Fort Pierce,5.1,6.0
1,Business,Fort Pierce,Fort Pierce,5.0,12.0
2,Business,Fort Pierce,Fort Pierce,4.8,13.0
3,Business,Fort Pierce,Fort Pierce,4.7,14.0
4,Business,Fort Pierce,West Palm Beach,63.7,67.0
...,...,...,...,...,...
1148,Business,Kar?chi,Kar?chi,4.6,23.0
1149,Business,Kar?chi,Kar?chi,0.8,4.0
1150,Business,Kar?chi,Kar?chi,0.7,7.0
1153,Business,Katunayake,Gampaha,6.4,18.0


Let's save our new data frame!

In [21]:
df.to_csv('New_UberDataset.csv', index=False)